In [25]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

from IPython.display import display, HTML

import papermill as pm
from configure_kgtk_notebooks import ConfigureKGTK

In [26]:
# Parameters

# Folder on local machine where to create the output and temporary folders
input_path = "/Users/pedroszekely/Downloads/kypher/iswc/inputs"
output_path = "/Users/pedroszekely/Downloads/kypher/iswc/ouptuts"
project_name = "arnold"
files = "all,label"

In [27]:
files = files.split(",")
ck = ConfigureKGTK()
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

In [28]:
os.environ['KGTK_LABEL_FILE'] = "{}".format(os.environ['label']) 
os.environ['kypher'] = "kgtk query --graph-cache /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold/wikidata.sqlite3.db"

In [29]:
ck.print_env_variables(files)

EXAMPLES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/examples
USE_CASES_DIR: /Users/pedroszekely/Documents/GitHub/kgtk/use-cases
GRAPH: /Users/pedroszekely/Downloads/kypher/iswc/inputs
OUT: /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold
TEMP: /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold
STORE: /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold/wikidata.sqlite3.db
kgtk: kgtk --debug
kypher: kgtk query --graph-cache /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold/wikidata.sqlite3.db
GRAPH: /Users/pedroszekely/Downloads/kypher/iswc/inputs
OUT: /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold
TEMP: /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold
STORE: /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold/wikidata.sqlite3.db
kgtk: kgtk --debug
kypher: kgtk query --graph-cache /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold/wikidata.sqlite3.db
GRAPH: /

In [30]:
ck.load_files_into_cache(file_list=files)

kgtk query --graph-cache /Users/pedroszekely/Downloads/kypher/iswc/ouptuts/arnold/temp.arnold/wikidata.sqlite3.db -i "/Users/pedroszekely/Downloads/kypher/iswc/inputs/all.tsv.gz" --as all  -i "/Users/pedroszekely/Downloads/kypher/iswc/inputs/labels.en.tsv.gz" --as label  --limit 3
node1	label	node2	id
P10	P31	Q18610173	P10-P31-Q18610173-85ef4d24-0
P1000	P31	Q18608871	P1000-P31-Q18608871-093affb5-0
P1001	P1647	P276	P1001-P1647-P276-e4e44f83-0


# To Do
- Do partition of the graph in to the usual Wikidata files
- Compute the derived files

# Explore The Arnold Schwarzenegger Graph `Q2685`



Number of nodes in the graph

In [31]:
!$kypher -i all \
--match '(n1)-[]->()' \
--return 'count(distinct n1)'

count(DISTINCT graph_1_c1."node1")
218486


Number of edges in the graph (not counting qualifier edges)

In [16]:
!$kypher -i all \
--match '()-[l]->()' \
--return 'count(distinct l)'

count(DISTINCT graph_1_c1."id")
1523735


Number of qualifier edges

In [17]:
!$kypher -i all \
--match '()-[l]->(), (l)-[q]->()' \
--return 'count(distinct q)'

count(DISTINCT graph_1_c2."id")
270275


Number of humans

In [33]:
!$kypher -i all \
--match '(n1)-[:P31]->(:Q5)' \
--return 'count(distinct n1)'

count(DISTINCT graph_1_c1."node1")
10918


Number of organizations (should be updated to use P279star)

In [18]:
!$kypher -i all \
--match '(n1)-[:P31]->(:Q43229)' \
--return 'count(distinct n1)'

count(DISTINCT graph_1_c1."node1")
206


Films where Schwarzenegger is a cast member

In [19]:
h = !$kypher -i all \
--match ' \
    (film)-[:P161]->(:Q2685)' \
--return 'distinct film as id' \
/ add-labels / html

display(HTML(h[0]))

id,id;label
Q110397,'True Lies'@en
Q15140437,'Terminator Genisys'@en
Q162255,'The Terminator'@en
Q170564,'Terminator 2: Judgment Day'@en
Q200804,'Predator'@en
Q222018,'Total Recall'@en
Q2842976,'American Masters'@en
Q29054009,'Terminator 3: Rise of the Machines'@en
Q309003,'Conan the Barbarian'@en
Q370326,'Eraser'@en


People who won the Bambi Award

In [31]:
%%time
h = !$kypher -i all \
--match ' \
    (film)-[:P166]->(:Q630018)' \
--return 'distinct film as id' \
/ add-labels / html

display(HTML(h[0]))

id,id;label
Q11975,'Britney Spears'@en
Q162389,'Tony Curtis'@en
Q212648,'Rudy Giuliani'@en
Q214574,'Jan Josef Liefers'@en
Q221074,'Bud Spencer'@en
Q229760,'Rita Ora'@en
Q243430,'Terence Hill'@en
Q2685,'Arnold Schwarzenegger'@en
Q312674,'Giorgio Moroder'@en
Q342617,'Ben Whishaw'@en


CPU times: user 3.42 ms, sys: 9.32 ms, total: 12.7 ms
Wall time: 1.15 s


Select the subgraph that contains item to item edges

In [15]:
%%time
!$kypher -i all \
--match ' \
    (n1)-[r {label: property}]->(n2), \
    (property)-[l:datatype]->(:`wikibase-item`)' \
--return 'distinct n1 as node1, property as label, n2 as node2, r as id' \
-o "$TEMP"/item.edges.tsv.gz

[2021-10-01 17:24:31 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", graph_1_c2."node1" "_aLias.label", graph_1_c1."node2" "_aLias.node2", graph_1_c1."id" "_aLias.id"
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_1 AS graph_1_c2
     ON graph_1_c2."node1" = graph_1_c1."label"
        AND graph_1_c1."label" = graph_1_c2."node1"
        AND graph_1_c2."label" = ?
        AND graph_1_c2."node2" = ?
  PARAS: ['datatype', 'wikibase-item']
---------------------------------------------
CPU times: user 84.2 ms, sys: 34.2 ms, total: 118 ms
Wall time: 7.69 s


Compute pagerank using the undirected graph

In [36]:
%%time
!$kgtk graph-statistics -i "$TEMP"/item.edges.tsv.gz \
--pagerank True \
--hits False \
--degrees False \
--undirected True \
--page-rank-property Ppagerank \
--statistics-only True \
-o "$TEMP"/pagerank.tsv.gz

!$kypher -i "$TEMP"/pagerank.tsv.gz --as pagerank --limit 2

node1	label	node2	id
P1001-P1855-Q11696-cdbf391b-0	vertex_in_degree	0	P1001-P1855-Q11696-cdbf391b-0-vertex_in_degree-0
P1001-P1855-Q11696-cdbf391b-0	vertex_out_degree	1	P1001-P1855-Q11696-cdbf391b-0-vertex_out_degree-1
CPU times: user 188 ms, sys: 73.5 ms, total: 261 ms
Wall time: 16.8 s


Compute degrees using the directed graph

In [37]:
%%time
!$kgtk graph-statistics -i "$TEMP"/item.edges.tsv.gz \
--pagerank False \
--hits False \
--degrees True \
--undirected False \
--vertex-in-degree-property Pindegree \
--vertex-out-degree-property Poutdegree \
--statistics-only True \
-o "$TEMP"/degrees.tsv.gz

!$kypher -i "$TEMP"/degrees.tsv.gz --as degrees --limit 2

node1	label	node2	id
P1001-P1855-Q11696-cdbf391b-0	Pindegree	0	P1001-P1855-Q11696-cdbf391b-0-Pindegree-0
P1001-P1855-Q11696-cdbf391b-0	Poutdegree	1	P1001-P1855-Q11696-cdbf391b-0-Poutdegree-1
CPU times: user 161 ms, sys: 65.2 ms, total: 227 ms
Wall time: 14.7 s


Top pagerank humans, go Stalone!

In [34]:
%%time
h=!$kypher -i all -i pagerank \
--match ' \
    all: (:Q5)<-[:P31]-(n1), \
    pagerank: (n1)-[l:Ppagerank]->(pagerank)' \
--return 'n1 as node1, l.label as label, pagerank as node2' \
--order-by 'cast(pagerank, float) desc' \
--limit 10 \
/ add-labels / html

display(HTML(h[0]))

node1,label,node2,node1;label
Q40026,Ppagerank,8.895744855562736e-05,'Sylvester Stallone'@en
Q55245,Ppagerank,8.60081254522788e-05,'Laurence Olivier'@en
Q42574,Ppagerank,8.053169474528737e-05,'James Cameron'@en
Q76,Ppagerank,7.859729524225644e-05,'Barack Obama'@en
Q2263,Ppagerank,7.455667246613304e-05,'Tom Hanks'@en
Q22686,Ppagerank,7.389881122758954e-05,'Donald Trump'@en
Q43203,Ppagerank,6.922374438747931e-05,'Clint Eastwood'@en
Q65932,Ppagerank,6.83331169060919e-05,'Anthony Hopkins'@en
Q35332,Ppagerank,6.772067757039425e-05,'Brad Pitt'@en
Q7542,Ppagerank,6.675128769339656e-05,'Prince'@en


CPU times: user 2.79 ms, sys: 8.17 ms, total: 11 ms
Wall time: 1.59 s


In degree

In [45]:
%%time
h=!$kypher -i all -i degrees \
--match ' \
    degrees: (n1)-[l:Pindegree]->(degree)' \
--return 'n1 as node1, l.label as label, degree as node2' \
--order-by 'cast(degree, int) desc' \
--limit 10 \
/ add-labels / html

display(HTML(h[0]))

node1,label,node2,node1;label
Q791801,Pindegree,16962,'estimation process'@en
Q30,Pindegree,13437,'United States of America'@en
Q5,Pindegree,10967,'human'@en
Q6581097,Pindegree,7193,'male'@en
Q39825,Pindegree,7177,'census'@en
Q1860,Pindegree,6899,'English'@en
Q15911027,Pindegree,3824,'demographic balance'@en
Q15221623,Pindegree,3178,'bilateral relation'@en
Q183,Pindegree,2984,'Germany'@en
Q28378282,Pindegree,2674,'verified account'@en


CPU times: user 4.02 ms, sys: 10.1 ms, total: 14.1 ms
Wall time: 1.32 s


Out degree

In [36]:
h=!$kypher -i all -i degrees \
--match ' \
    degrees: (n1)-[l:Poutdegree]->(degree), all: (n1)-[:P31]->(:Q5)' \
--return 'n1 as node1, l.label as label, degree as node2' \
--order-by 'cast(degree, int) desc' \
--limit 10 \
/ add-labels / html

display(HTML(h[0]))

node1,label,node2,node1;label
Q8704,Poutdegree,98,'Walt Disney'@en
Q22686,Poutdegree,87,'Donald Trump'@en
Q23505,Poutdegree,85,'George H. W. Bush'@en
Q76,Poutdegree,85,'Barack Obama'@en
Q9916,Poutdegree,81,'Dwight D. Eisenhower'@en
Q38111,Poutdegree,73,'Leonardo DiCaprio'@en
Q103876,Poutdegree,72,'Peter O\'Toole'@en
Q9960,Poutdegree,71,'Ronald Reagan'@en
Q2105,Poutdegree,69,'Jacques Chirac'@en
Q9582,Poutdegree,68,'Gerald Ford'@en


q-nodes with ULAN id

In [21]:
!$kypher -i all \
--match '(n1)-[:P245]->(ulan_id)' \
--return 'count(distinct n1)'

[2021-10-01 17:25:02 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_1_c1."node1")
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = ?
  PARAS: ['P245']
---------------------------------------------
count(DISTINCT graph_1_c1."node1")
431


Sample of nodes with ULAN ids

In [22]:
!$kypher -i all \
--match '(n1)-[l:P245]->(ulan_id)' \
--return 'distinct n1 as node1, l.label as label, ulan_id as node2' \
--limit 10 \
/ add-labels / table

[2021-10-01 17:25:03 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", graph_1_c1."label" "_aLias.label", graph_1_c1."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = ?
     LIMIT ?
  PARAS: ['P245', 10]
---------------------------------------------
| node1     | label | node2       | node1;label                            | label;label                        |
| --------- | ----- | ----------- | -------------------------------------- | ---------------------------------- |
| Q100948   | P245  | "500224955" | 'Rachel Carson'@en                     | 'Union List of Artist Names ID'@en |
| Q101771   | P245  | "500281177" | 'Gottfried Gruben'@en                  | 'Union List of Artist Names ID'@en |
| Q101791   | P245  | "500001235" | 'Sep Ruf'@en                           | 'Union List of Artist Names ID'@en |
| Q102139   | P245  | "500256782" | 'Margrethe II of Denmark'@en     

Count of q-nodes with ULAN ids and place of birth (P19)

In [23]:
!$kypher -i all \
--match '(n1)-[:P245]->(ulan_id), (n1)-[:P19]->()' \
--return 'count(distinct n1)'

[2021-10-01 17:25:03 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_1_c1."node1")
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_1 AS graph_1_c2
     ON graph_1_c1."node1" = graph_1_c2."node1"
        AND graph_1_c1."label" = ?
        AND graph_1_c2."label" = ?
  PARAS: ['P245', 'P19']
---------------------------------------------
count(DISTINCT graph_1_c1."node1")
143


Sample of q-nodes that have ULAN ids, but don't have birth place

In [24]:
!$kypher -i all \
--match '(n1)-[:P245]->(ulan_id)' \
--opt '(n1)-[:P19]->(birth_place)' \
--where: 'birth_place is null' \
--return 'distinct n1 as node1' \
--limit 10 \
/ add-labels / table

[2021-10-01 17:25:04 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1"
     FROM graph_1 AS graph_1_c1
     LEFT JOIN graph_1 AS graph_1_c2
     ON graph_1_c1."node1" = graph_1_c2."node1"
        AND graph_1_c2."label" = ?
     WHERE graph_1_c1."label" = ?
        AND (graph_1_c2."node2" IS NULL)
     LIMIT ?
  PARAS: ['P19', 'P245', 10]
---------------------------------------------
| node1     | node1;label                            |
| --------- | -------------------------------------- |
| Q100948   | 'Rachel Carson'@en                     |
| Q102139   | 'Margrethe II of Denmark'@en           |
| Q1024362  | 'Spanish National Research Council'@en |
| Q1024426  | 'University of South Carolina'@en      |
| Q102711   | 'Dennis Hopper'@en                     |
| Q10288082 | 'Wildenstein & Company'@en             |
| Q1065     | 'United Nations'@en                    |
| Q1065414  | 'Vrije Universiteit Amsterdam'@e

Count of q-nodes with ULAN id and birth date

In [25]:
!$kypher -i all \
--match '(n1)-[:P245]->(ulan_id), (n1)-[:P569]->()' \
--return 'count(distinct n1)'

[2021-10-01 17:25:05 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_1_c1."node1")
     FROM graph_1 AS graph_1_c1
     INNER JOIN graph_1 AS graph_1_c2
     ON graph_1_c1."node1" = graph_1_c2."node1"
        AND graph_1_c1."label" = ?
        AND graph_1_c2."label" = ?
  PARAS: ['P245', 'P569']
---------------------------------------------
count(DISTINCT graph_1_c1."node1")
240


Sample of q-nodes that have ULAN ids, but don't have birth date (P569)

In [38]:
!$kypher -i all \
--match '(n1)-[:P245]->(ulan_id), (n1)-[:P31]->(:Q5)' \
--opt '(n1)-[:P569]->(birth_place)' \
--where: 'birth_place is null' \
--return 'distinct n1 as node1' \
--limit 10 \
/ add-labels / table

| node1    | node1;label            |
| -------- | ---------------------- |
| Q1280275 | 'Pan Painter'@en       |
| Q133337  | 'Solon'@en             |
| Q1405    | 'Augustus'@en          |
| Q168261  | 'Ptolemy I Soter'@en   |
| Q2039    | 'Titus Livius'@en      |
| Q26825   | 'Herodotos'@en         |
| Q43353   | 'Aristophanes'@en      |
| Q5264    | 'Hippocrates'@en       |
| Q577906  | 'Antimenes Painter'@en |
| Q902022  | 'Darius Painter'@en    |


In [27]:
!$kypher -i all \
--match '(n1)-[l:P245]->(ulan_id)' \
--return 'distinct n1 as node, l.label as label, ulan_id as node2' \
-o "$TEMP"/ulan-qnodes.tsv

[2021-10-01 17:25:07 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node", graph_1_c1."label" "_aLias.label", graph_1_c1."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label" = ?
  PARAS: ['P245']
---------------------------------------------
